In [ ]:
import tensorflow as tf
import numpy as np
from binance.client import Client
from binance.exceptions import BinanceAPIException
from keys import api_key, secret_key

In [ ]:
# Set the name of the model to load
model_name = input("Enter the name of the model to load: ")

In [ ]:
# Set up the Binance client
client = Client(api_key, secret_key, tld="us")

In [ ]:
# Load the saved model
model = tf.keras.models.load_model("models/{}".format(model_name))

In [ ]:
symbol = "BTC" # CHANGE TARGET COIN HERE
# EXPECT INSUFFICIENT FUNDS ; CHECK COIN TRADE REQUIREMENTS HERE https://www.binance.com/en/trade-rule

In [ ]:
# Define a function to preprocess market data
def preprocess(market_data):
    # Retrieve historical price data from the Binance API
    symbol = market_data['symbol']
    timeframe = market_data['timeframe']
    bars = market_data['bars']
    klines = client.get_klines(symbol=symbol, interval=timeframe, limit=max(bars, 60))
    # Extract the open, high, low, and close prices from the klines data
    prices = np.array([[float(kline[1]), float(kline[2]), float(kline[3]), float(kline[4])] for kline in klines])
    # Normalize the prices
    prices_norm = (prices - np.mean(prices, axis=0)) / np.std(prices, axis=0)
    # Reshape the prices for the model input shape
    if len(prices_norm) >= 60:
        prices_reshaped = prices_norm[-60:].reshape(-1, 60, 4)
    else:
        prices_reshaped = np.zeros((1, 60, 4))
    # Return the preprocessed data
    return prices_reshaped

In [ ]:
# Define a function to postprocess model predictions
def postprocess(predictions):
    # Return the strength of the predicted market movement
    return round(float(predictions[0][0]), 3)

In [ ]:
# Define a function to make predictions on live market data and create market orders
def make_predictions_and_create_orders(market_data):
    # Make predictions using the loaded model
    predictions = model.predict(preprocess(market_data))
    # Postprocess the predictions to get the strength of the predicted market movement
    market_movement_strength = postprocess(predictions)
    # Determine the direction of the predicted market movement
    if market_movement_strength > 0:
        direction = 'BUY'
    else:
        direction = 'SELL'
    # Determine the size of the market order based on the strength of the prediction
    size = float(abs(market_movement_strength) / 100) # Adjust 100 as needed to match your trading amount
    size_str = "{:.6f}".format(size)
    print(size_str)
    # Create the market order
    try:
        order = client.create_order(symbol=market_data['symbol'], side=direction, type='MARKET', quantity=size_str)
        print(f'Market order created: {order}')
    except BinanceAPIException as e:
        print(f'Error creating market order: {e}')

In [ ]:
market_data = {
    'symbol': 'BTCUSDT', # CHANGE TARGET COIN HERE
    'timeframe': '1h',
    'bars': 24 # Predict on the last 24 hours of price data
}
make_predictions_and_create_orders(market_data)